In [3]:
import numpy as np
from scipy.stats import norm, uniform

In [74]:
num_simulations = 10000
num_periods = 60
face_value = 100000
coupon_rate = 0.07
recovery_rate = 0.4
correlation = 0.25
copula_params = 2.0  # Clayton copula parameter
risk_free_rate = 0.035  # 3.5%

In [21]:
def simulate_default_probabilities(num_simulations, num_periods, correlation, copula_params):
    # Generate independent standard normal random variables
    independent_normals = np.random.normal(size=(num_simulations, num_periods))

    # Generate correlated random variables using the Clayton copula
    correlated_normals = copula_transform(independent_normals, correlation, copula_params)

    # Transform correlated normals to uniform marginals
    uniform_marginals = norm.cdf(correlated_normals)

    # Simulate default probabilities based on the uniform marginals
    default_probabilities = 1 - uniform_marginals

    return default_probabilities

def copula_transform(normals, rho, copula_params):
    """
    Clayton copula transformation function.
    """
    num_columns = normals.shape[1]
    transformed_variables = np.empty_like(normals)

    for i in range(num_columns-1):
        u = normals[:, i]
        v = normals[:, i + 1] if i + 1 < num_columns else normals[:, 0]  # Wrap around to the first column for the last variable

        # Clayton copula transformation formula
        w = (u**(-rho) + v**(-rho) - 1)**(-1/rho)

        transformed_variables[:, i] = u
        transformed_variables[:, i + 1] = w if i + 1 < num_columns else w  # Wrap around

    return transformed_variables



In [20]:

default_probabilities = simulate_default_probabilities(num_simulations, num_periods, correlation, copula_params)

# Print the first 5 simulated default probabilities for each period
print("Simulated Default Probabilities:")
print(default_probabilities[:1, :])
print(default_probabilities.shape)

Simulated Default Probabilities:
[[0.50245672 0.57643122 0.29225016 0.81446907 0.14022413 0.97189996
  0.6102696  0.69755281 0.34284077 0.93935801 0.01143382 0.00247391
  0.62276436 0.20073539 0.56040238 0.00429667 0.62067326 0.14417212
  0.49894754 0.09901783 0.91403676 0.3329517  0.12226871 0.95311712
  0.24499758 0.80468837 0.29698929 0.13693157 0.91628069 0.6082653
  0.57384038 0.16634784 0.59402693 0.19058888 0.47552467 0.20929502
  0.28485139 0.44812964 0.30373663 0.5338361  0.65711457 0.80025779
  0.90448769 0.56050068 0.28678862 0.56660976 0.67204596 0.64358098
  0.02113247 0.01329183 0.53693818 0.67236278 0.71767071 0.24612022
  0.7782153  0.74302425 0.17622339 0.03038525 0.62743617        nan]]
(10000, 60)


/var/folders/xc/6c2pzz2172x4x83lp41y7csm0000gn/T/ipykernel_18800/270410040.py:28: RuntimeWarning: invalid value encountered in power
  w = (u**(-rho) + v**(-rho) - 1)**(-1/rho)


In [44]:
def generate_default_scenarios(default_probabilities):
    """
    Generate default scenarios based on simulated default probabilities.
    """
    num_simulations, num_periods = default_probabilities.shape

    # Compare default probabilities to determine defaults
    default_scenarios = 0.9 < default_probabilities

    return default_scenarios

In [45]:
default_scenarios = generate_default_scenarios(default_probabilities)

In [46]:
# Print the first 5 default scenarios for each period
print("Default Scenarios:")
print(default_scenarios[:5, :].astype(int))
print(default_scenarios.shape)

Default Scenarios:
[[0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0
  0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
  0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
  0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
  0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0]]
(10000, 60)


In [47]:
def first_one(A):
    if len(A) == 0:
        return -1

    return first_one_subarray(A, 0, len(A) - 1)

def first_one_subarray(A, start, end):
    # Incorrect subarray
    if start > end or start > len(A) - 1:
        return -1

    # Check if 1 is on 'first' position
    if A[start] == 1:
        return start

    # Divide into two parts
    split_point = max(start + 1, round(end / 2))
    result_left = first_one_subarray(A, start + 1, split_point)
    result_right = first_one_subarray(A, split_point + 1, end)

    if result_left != -1:
        return result_left
    else:
        return result_right

In [48]:
default_times = []
for i in range(60):
    default_times.append(first_one(default_scenarios[i,:]))

In [50]:
# Initialize CDO values
cdo_values = np.zeros(num_simulations)

In [72]:
def calculate_cdo_cash_flows(default_scenarios, face_value, coupon_rate, recovery_rate):
    """
    Calculate CDO cash flows based on default scenarios.
    """
    num_simulations, num_periods = default_scenarios.shape

    # Initialize CDO cash flows matrix
    cdo_cash_flows = np.zeros((num_simulations, num_periods))
    #print(cdo_cash_flows[1,:])
    i=0
    for t in range(num_simulations):
        # Calculate cash flows for each simulation at time t
        flag=True
        i=0
        for i in range(num_periods):
            #print(flag)
            if flag:
                if default_scenarios[t, i] > 0.9:  # Bond defaulted
                    cdo_cash_flows[t, i] = face_value * (1 - recovery_rate)  # Loss given default
                    flag = False
                else:  # Bond did not default
                    cdo_cash_flows[t, i] = face_value * coupon_rate / 2  # 6-month coupon payment

    return cdo_cash_flows

In [73]:
cdo_cash_flows = calculate_cdo_cash_flows(default_scenarios, face_value, coupon_rate, recovery_rate)

# Print the first 5 rows of the calculated CDO cash flows
print("CDO Cash Flows:")
print(cdo_cash_flows[:5, :])

CDO Cash Flows:
[[ 3500.  3500.  3500.  3500.  3500. 60000.     0.     0.     0.     0.
      0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
      0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
      0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
      0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
      0.     0.     0.     0.     0.     0.     0.     0.     0.     0.]
 [60000.     0.     0.     0.     0.     0.     0.     0.     0.     0.
      0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
      0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
      0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
      0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
      0.     0.     0.     0.     0.     0.     0.     0.     0.     0.]
 [ 3500.  3500.  3500.  3500.  3500.  3500.  3500.  3500.  3500.  3500.
   3500.  3500.  3500.  3500.  3500.  3500.  3

In [75]:
def discount_cash_flows(cdo_cash_flows, risk_free_rate):
    """
    Discount CDO cash flows to present value using the risk-free rate.
    """
    num_simulations, num_periods = cdo_cash_flows.shape

    # Initialize discounted cash flows matrix
    discounted_cash_flows = np.zeros((num_simulations, num_periods))

    for t in range(num_periods):
        discount_factor = 1 / (1 + risk_free_rate / 2) ** (t + 1)  # 6-month compounding

        # Discount cash flows for each simulation at time t
        discounted_cash_flows[:, t] = cdo_cash_flows[:, t] * discount_factor

    return discounted_cash_flows

In [76]:
discounted_cash_flows = discount_cash_flows(cdo_cash_flows, risk_free_rate)

In [93]:
# Print the first 5 rows of the discounted cash flows
print("Discounted Cash Flows:")
print(discounted_cash_flows[:1, :])

Discounted Cash Flows:
[[ 3439.8034398   3380.64220128  3322.49847792  3265.35476945
   3209.19387661 54068.5525017      0.             0.
      0.             0.             0.             0.
      0.             0.             0.             0.
      0.             0.             0.             0.
      0.             0.             0.             0.
      0.             0.             0.             0.
      0.             0.             0.             0.
      0.             0.             0.             0.
      0.             0.             0.             0.
      0.             0.             0.             0.
      0.             0.             0.             0.
      0.             0.             0.             0.
      0.             0.             0.             0.
      0.             0.             0.             0.        ]]


In [80]:
discounted_cash_flows.shape

(10000, 60)

In [87]:
for i in range(num_simulations):
    cdo_values[i] = np.sum(discounted_cash_flows[i,:])

In [88]:
cdo_values.shape

(10000,)

In [89]:
expected_cdo_value = np.mean(cdo_values)

In [90]:
expected_cdo_value

77873.14763644773

In [91]:
cdo_values

array([ 70686.04526677,  58968.05896806,  94989.58400639, ...,
        66120.45445247, 108597.71862391,  87443.63723783])

In [92]:
discounted_cash_flows

array([[ 3439.8034398 ,  3380.64220128,  3322.49847792, ...,
            0.        ,     0.        ,     0.        ],
       [58968.05896806,     0.        ,     0.        , ...,
            0.        ,     0.        ,     0.        ],
       [ 3439.8034398 ,  3380.64220128,  3322.49847792, ...,
            0.        ,     0.        ,     0.        ],
       ...,
       [ 3439.8034398 ,  3380.64220128,  3322.49847792, ...,
            0.        ,     0.        ,     0.        ],
       [ 3439.8034398 ,  3380.64220128,  3322.49847792, ...,
            0.        ,     0.        ,     0.        ],
       [ 3439.8034398 ,  3380.64220128,  3322.49847792, ...,
            0.        ,     0.        ,     0.        ]])